In [222]:
from mnist import MNIST 

mndata = MNIST('./data')

mndata.load_training()
img_side = 28
n_mnist_pixels = img_side*img_side

In [231]:
import numpy as np

layer_size = 1024

I = np.zeros(n_mnist_pixels)
O = np.zeros(layer_size)
W = np.random.rand(O.size, I.size) * 0.001

In [232]:
push_delta = 0.4
k = 1

In [233]:
num_samples = 1
i_idx = 10

img = mndata.train_images[i_idx]

# infer
I = np.array(img) / 255
O = np.dot(W, I)

# train
dW = I - (W * O[np.newaxis].T)
wO = np.argsort(O)[::-1]
O.fill(0)
O[wO[0]] = 1
O[wO[1:1+k]] = -push_delta
dW *= O[np.newaxis].T
W.shape

(1024, 784)

In [234]:
syn_n = 1.5

syn_size = W.shape[1]
W_idx = np.arange(syn_size)
W_idx.shape
W_len = np.abs(W_idx[np.newaxis].T - W_idx)
W_len = (np.array(W_len, dtype='float') + 1) ** -syn_n

W_len_1 = np.copy(W_len)
W_len_1 = np.expand_dims(W_len_1, axis=0)
print(W_len_1.shape)
print(W.shape)
print(W.T.shape, W_len[np.newaxis].T.shape, (W.T * W_len[np.newaxis].T).shape, (W.T * W_len[np.newaxis].T).T.shape)
res_sum =  np.sum((W.T * W_len[np.newaxis].T).T, axis=1)

# (W.T.shape, W_len.T.shape)

# W_2 = np.expand_dims(W, axis=1)
# print(W_2.shape, W_len_1.shape)
print(W.T.shape, W_len_1.T.shape)
res_1 = np.tensordot(W, W_len_1, axes=(1,1)) #.reshape(W.shape[0], W.shape[1])

print(res_1.shape, res_sum.shape)
# print(res_sum[0] - res_1[0][0])
# print(res_1[0][0][1])
# np.dot(W, W_len)

# np.fill_diagonal(a_len, 0)
# d_a = np.dot(W, a_len)

(1, 784, 784)
(1024, 784)
(784, 1024) (784, 784, 1) (784, 784, 1024) (1024, 784, 784)
(784, 1024) (784, 784, 1)
(1024, 1, 784) (1024, 784)


In [235]:
W_len_2 = np.copy(W_len)

dW_2 = np.zeros(W.shape)
for n_idx in range(W.shape[0]):
    dW_2[n_idx] = np.dot(W[n_idx], W_len_2)
np.sum(dW_2 - res_1)


-1.1498413332589053e-11

In [239]:
r = np.sum((W.T * W_len[np.newaxis].T).T, axis=1)

In [311]:
for n_idx in range(W.shape[0]):
    dW_2[n_idx] = np.dot(W[n_idx], W_len_2)

In [312]:
res_dot = np.dot(W, W_len).reshape(W.shape[0],W.shape[1])

In [313]:
np.abs(dW_2 - res_dot).sum()

9.705861331657517e-13

In [314]:
res_tdot = np.tensordot(W, W_len, axes=(1,1)).reshape(W.shape[0],W.shape[1])

In [315]:
np.abs(dW_2 - res_tdot).sum()

9.705861331657517e-13